# A1: Logic and lambda calculus

Simon Dobnik and Robin Cooper

The lab is an exploration and learning exercise to be done in a group and also in discussion with the teachers and other students.

Before starting, please read the instructions on how to work in groups on Canvas.

Write all your answers and the code in the appropriate boxes below.

*Important*: We recommend that you create a virtual environment (either with virtualenv or conda), install Jupyter Lab in it and all dependencies used in the assignment.
To run Jupyter Lab later within your environment, run the following command:

```python -m ipykernel install --user --name=my-virtualenv-name```,

where you replace `my-virtualenv-name` with the name of your created environment.
Once in Jupyter, choose the kernel with the name of your environment. You can do it by either (i) using the drop-down menu in the top right corner or (ii) going to the top menu -> Kernel -> Change Kernel.

## Translating English to logic and evaluating logic in a model

In [1]:
# This task needs NLTK and Jupyter Lab (IPython package).
import nltk
from utils import display_latex, display_translation, display_tree, display, Markdown
read_expr = nltk.sem.Expression.fromstring

### 1. Propositional logic
Translate the following sentences into **propositional logic** and verify that they parse with Expression.fromstring() (`read_expr` variable in the cell above). Provide a key which shows how the propositional variables in your translation correspond to expressions of English. Briefly discuss any difficulties you encounter. (By difficulties we mean cases where the semantics of English expressions cannot be expressed to the same degree by the semantics of your logic representations, i.e. they do not mean the same). **[5 + 1 marks]**

In [4]:
# Define keys

key = {
    "A": "Alex plays the piano.",
    "B": "Alex is smart.",
    "C": "Alex is musical.",
    "D": "Lydia is happy.",
    "E": "George plays the piano.",
    "F": "George is musical.",
}

propositions = {
    "If Alex plays the piano, she is smart.": read_expr('A -> B'),
    "Alex is both smart and musical.": read_expr('B & C'),
    "If Alex is not smart, Lydia is not happy.": read_expr('-B -> -D'),
    # First interpretation
    # If either one plays the piano, they are both musical
    "If Alex or George plays the piano, they are musical.": read_expr('(A | E) ->(C & F)'),
    #Second interpretation
    # If either one plays the piano, then the one that plays it's musical
    "If Alex or George plays the piano, then they are musical.": read_expr('(A -> C ) | (E->F)'),
    "George plays the piano.": read_expr('E'),
}


for text, semrep in propositions.items():
    display_translation(text, semrep)

"If Alex plays the piano, she is smart.": $(A\ \rightarrow\ B)$

"Alex is both smart and musical.": $(B\ \land\ C)$

"If Alex is not smart, Lydia is not happy.": $(-B\ \rightarrow\ -D)$

"If Alex or George plays the piano, they are musical.": $((A\ \lor\ E)\ \rightarrow\ (C\ \land\ F))$

"If Alex or George plays the piano, then they are musical.": $((A\ \rightarrow\ C)\ \lor\ (E\ \rightarrow\ F))$

"George plays the piano.": $E$

## Difficulties encountered:

### Sentence 1
>*If Alex plays the piano, she is smart.*

Since propositional logic treats sentences as atomic symbols, it cannot represente coreference, the link between *Alex* and *she*. Hence we manually map *she* to *Alex* using keys.

### Sentence 2
>*Alex is both smart and musical.*

Translated as conjuntion (B & C), however the emphasis on "both" is lost.

### Sentence 4
>*If Alex or George plays the piano, they are musical.*

Two readings are possible (ignoring the unlikely reading from a context where "Alex or George" is a music band). It's ambiguity arises due to the fact that there is no atomic reference to either Alex or George, no quantification over them, and no distinction between singular and plural.

**First interpretation**

>*If Alex or George plays the piano, they are musical. (if either one does, they both are musical).*

This first translation preserves the structure of the original sentence, and refers to *they* as both Alex and George as a unit. However, this interpretation it's restrictive, demanding that both *Alex* and *George* be musical even if only one of them plays, which might not represent the more natual distributive reading.

**Second interpretation**
>*If Alex or George plays the piano, they are musical. (if either one does, then that one is musical).*

This interpretation handles distributivity better than the first interpretation, however it does not preserves the structure of the original sentence.




### 2. Valuation of Propositional logic

Imagine that we observe a world where
- (i) Alex does not play the piano,
- (ii) Alex and Lydia are smart and musical,
- (iii) George is not musical,
- (iv) Lydia is happy,
- (v) George plays the piano.

Translate this informal description of the world into a model by appropriately defining an evaluation function and evaluate the formulae from Question 1 in this model. Briefly comment the answers you get. **[5 + 1 marks]**.

In [5]:


#Mapping of the informal descriptions
key2 = {
    "A": "Alex plays the piano",
    "B": "Alex is smart",
    "C": "Alex is musical",
    "D": "Lydia is smart",
    "E": "Lydia is musical",
    "F": "George is musical",
    "G": "George plays the piano",
    "H": "Lydia is happy"
}


val = nltk.Valuation([
    ("A", False),
    ("B", True),
    ("C", True),
    ("D", True),
    ("E", True),
    ("F", False),
    ("G", True),
    ("H", True),
])

dom = set()
g = nltk.Assignment(dom)
m = nltk.Model(dom, val)


print("Exercise 2: Evaluation Results")
for text, semrep in propositions.items():
    result = m.evaluate(str(semrep), g)
    print(f"{text}\n→ {result}\n")
    display_latex(str(semrep))


Exercise 2: Evaluation Results
If Alex plays the piano, she is smart.
→ True



$(A\ \rightarrow\ B)$

Alex is both smart and musical.
→ True



$(B\ \land\ C)$

If Alex is not smart, Lydia is not happy.
→ True



...

$((A\ \lor\ E)\ \rightarrow\ (C\ \land\ F))$

If Alex or George plays the piano, then they are musical.
→ True



$((A\ \rightarrow\ C)\ \lor\ (E\ \rightarrow\ F))$

George plays the piano.
→ True



$E$

##Comments:

### Sentence 1
>*If Alex plays the piano, she is smart.*

This sentence exemplifies the idea of **vacuous truth**. Due to how implication works in logic, and the fact that an implication is *False* only when the first part is *True* and the second is *False*, this sentence ends up evaluating to *True*, even though the first part is *False*. However, in real-world situations, if Alex playing the piano and her being smart are directly dependent, the implication might be considered *False* if she does not play.

### Sentence 4
>*Alex or George plays the piano, they are musical.*

For this sentence, we can see that the idea of singular (interpretation 2) vs. plural (interpretation 1) turned out to be semantically relevant, as the second interpretation evaluates to *True*, but the first to *False*. As previously noted, although the first interpretation conserves the structure of the sentence better, it is more restrictive, as it needs both conditions in the consequent to be true.


### 3. Predicate logic *without quantifiers*

Translate the following sentences into predicate-argument formulae of First Order Logic and verify that they parse with `Expression.fromstring()`. Briefly discuss any difficulties you encounter. **[4 + 1 marks]**

In [6]:
sentences1 = {
    "Lydia likes George but Lydia doesn't like Alex":
    read_expr(r'like(lydia, george) & -like(lydia, alex)'),

    "Lydia likes herself and so does George (George also likes himself)":
    read_expr(r'like(lydia, lydia) & like(george, george)'),

    "Lydia likes herself and so does George":
    read_expr(r'like(lydia, lydia) & like(george, lydia)'),


    "Charlie is an English pianist who plays a sonata":
    read_expr(r'english(charlie) & pianist (charlie) & play(charlie, sonata)'),

    "Lydia and George admire each other":
    read_expr(r'admire(lydia, george) & admire (george, lydia)'),
}

for text, semrep in sentences1.items():
    display_translation(text, semrep)


"Lydia likes George but Lydia doesn't like Alex": $(like(lydia,george)\ \land\ -like(lydia,alex))$

"Lydia likes herself and so does George (George also likes himself)": $(like(lydia,lydia)\ \land\ like(george,george))$

"Lydia likes herself and so does George": $(like(lydia,lydia)\ \land\ like(george,lydia))$

"Charlie is an English pianist who plays a sonata": $(english(charlie)\ \land\ pianist(charlie)\ \land\ play(charlie,sonata))$

"Lydia and George admire each other": $(admire(lydia,george)\ \land\ admire(george,lydia))$

## Difficulties encountered:

### Sentence 1
>*Lydia likes George but Lydia doesn't like Alex.*

FOL cannot directly represent contrastive conjunction. For this sentence, we had to use the logical conjunction *and*. Though this change does not seem to bring much difference in meaning, we do lose the implied contrast between the two clauses.


### Sentence 2
>*Lydia likes herself and so does George.*

The phrase *so does George* is ambigous as it could mean both *George likes Lydia as well* and *George likes himself*.

**First interpretation**
> *Lydia likes herself and so does George (George also likes himself).*

**Second interpretation**
>*Lydia likes herself and so does George (George also likes Lydia).*


### Sentence 3
>*Charlie is an English pianist who plays a sonata.*

Since we were asked to represent the sentences without the use of quantifiers, the NP *a sonata* is represented as a constant. As a result, we refer to a specific, named sonata, which the sentence does not actually imply. The English sentence implies existence, but not uniqueness.





### 4. First order logic with quantifiers

Translate the following sentences into quantified formulas of First Order Logic and verify that they parse with `Expression.fromstring()`. Briefly discuss any difficulties you encounter. **[4 + 1 marks]**

In [16]:
sentences2 = {
    "Charlie knows a woman who likes George":
    read_expr('exists x.(women(x) & know(charlie,x) & like(x, george))'),

    "George admires everybody and Lydia admires nobody":
    read_expr('all x.(admire(george, x) & -admire(lydia, x) )'),

    "Nobody admires everybody (domain neutral)":
    read_expr('-exists x. all y. admire(x, y)'),

    "Nobody admires everybody":
    read_expr('not exist x.(all y.((woman(x) or man(x)) -> admire(x, y)))'),

    "Exactly one musician plays everything Alex wrote (simple)":
    read_expr('exists x.(all y. ((musician(x) and write(alex, y)) -> play(x, y)))'),

    "Exactly one musician plays everything Alex wrote (There is a musician that plays everything Alex wrote, and for all other musicians there is nothing such that Alex wrote and they play it.)":
    read_expr('exists x.(musician(x) & all y. (write(alex, y) -> play(x, y)) -> (all z. (musician(z) & -(z=x) -> -exists w.(write(alex, w) & play(z, w)))))'),

    "Exactly one musician plays everything Alex wrote (There is a musician that plays everything Alex wrote, such that all other musicians that play everything Alex wrote are that musician)":
    read_expr('exists x.(all y.((((musician(x) and write(alex, y)) -> play(x, y))) -> (all z.((musician(z) and write(alex, y) and play(z, y) -> z=x)))))'),
}

for text, semrep in sentences2.items():
    display_translation(text, semrep)

"Charlie knows a woman who likes George": $\exists\ x.(women(x)\ \land\ know(charlie,x)\ \land\ like(x,george))$

"George admires everybody and Lydia admires nobody": $\forall\ x.(admire(george,x)\ \land\ -admire(lydia,x))$

"Nobody admires everybody (domain neutral)": $-\exists\ x.\forall\ y.admire(x,y)$

"Nobody admires everybody": $-\exists\ x.\forall\ y.((woman(x)\ \lor\ man(x))\ \rightarrow\ admire(x,y))$

"Exactly one musician plays everything Alex wrote (simple)": $\exists\ x.\forall\ y.((musician(x)\ \land\ write(alex,y))\ \rightarrow\ play(x,y))$

"Exactly one musician plays everything Alex wrote (There is a musician that plays everything Alex wrote, and for all other musicians there is nothing such that Alex wrote and they play it.)": $\exists\ x.((musician(x)\ \land\ \forall\ y.(write(alex,y)\ \rightarrow\ play(x,y)))\ \rightarrow\ \forall\ z.((musician(z)\ \land\ -(z\ =\ x))\ \rightarrow\ -\exists\ w.(write(alex,w)\ \land\ play(z,w))))$

"Exactly one musician plays everything Alex wrote (There is a musician that plays everything Alex wrote, such that all other musicians that play everything Alex wrote are that musician)": $\exists\ x.\forall\ y.(((musician(x)\ \land\ write(alex,y))\ \rightarrow\ play(x,y))\ \rightarrow\ \forall\ z.((musician(z)\ \land\ write(alex,y)\ \land\ play(z,y))\ \rightarrow\ (z\ =\ x)))$

## Difficulties encountered:

### Sentence 3
>*Nobody admires everybody.*

The translation of this sentence into logics depends on the domain of discourse, considering that we do not have yet a domain of discouse, we could have two different interpretations:

**First interpretation**
In this interpretation *nobody* and *everybody* can represent anything, from humans to robots.

**Second interpretation**
In this interpretation *nobody* and *everybody* refers only to human, women and men,and we therefore apply predicates that specify whether an entity is a woman or a man

### Sentence 4
>*Exactly one musician plays everything Alex wrote.*

The challenge of encoding the last sentence constitutes choosing a way to express "exactly one", either negatively, that no other musician fulfill the criterion of playing everything Alex wrote, or positively, that any other musicians that fulfill the conditions are the same entitiy.




In [17]:
entities = set(['l','g','a','c','b','s','u','e','p','w'])

assign = """
  lydia => l
  george => g
  alex => a
  charlie => c
  bertie => b
  sonata => s
  etude => e
  prelude => p
  waltz => w
  scherzo => u


  like=>{(l,l), (l,g), (l,a), (l,c), (g,l), (g,b), (g,g), (a,a), (c,l), (c,g), (c,a), (c,c), (c,b), (b,a)}
  english=>{l, g, a, c, b}
  pianist=>{c, b}
  play=>{(c,s), (c,e), (c,w), (b,w), (b,u), (l,e), (l,p), (l,w)}
  admire=>{(l,l), (l,c), (l,b), (g,l), (g,g), (g,a), (g,c), (g,b), (a,l), (a,a), (a,b), (c,g), (c,b), (b,l), (b,g), (b,a), (b,c), (b,b)}
  know=>{(l,l), (l,g), (l,a), (l,c), (l,b), (g,l), (g,g), (g,b), (a,l), (a,a), (a,b), (c,g), (c,c), (c,b), (b,l), (b,g), (b,a), (b,c), (b,b)}
  woman=>{l, a, c}
  man=>{g, b}
  write=>{(a,s), (a,e), (a,w)}
  musician=>{l, a, c, b}

"""

val2 = nltk.Valuation.fromstring(assign)

g2 = nltk.Assignment(entities)
m2 = nltk.Model(entities, val2)

# Sentences from question 3
for text, semrep in sentences1.items():
    print(text)
    print(m2.evaluate(str(semrep), g2))
    display_latex(semrep)
    display(Markdown('----'))

print ()
print("Sentences from question 4")
print()
# Sentences from question 4
for text, semrep in sentences2.items():
    print(text)
    print(m2.evaluate(str(semrep), g2))
    display_latex(semrep)
    display(Markdown('----'))


Lydia likes George but Lydia doesn't like Alex
False


$(like(lydia,george)\ \land\ -like(lydia,alex))$

----

Lydia likes herself and so does George (George also likes himself)
True


$(like(lydia,lydia)\ \land\ like(george,george))$

...

$\exists\ x.((musician(x)\ \land\ \forall\ y.(write(alex,y)\ \rightarrow\ play(x,y)))\ \rightarrow\ \forall\ z.((musician(z)\ \land\ -(z\ =\ x))\ \rightarrow\ -\exists\ w.(write(alex,w)\ \land\ play(z,w))))$

----

Exactly one musician plays everything Alex wrote (There is a musician that plays everything Alex wrote, such that all other musicians that play everything Alex wrote are that musician)
True


$\exists\ x.\forall\ y.(((musician(x)\ \land\ write(alex,y))\ \rightarrow\ play(x,y))\ \rightarrow\ \forall\ z.((musician(z)\ \land\ write(alex,y)\ \land\ play(z,y))\ \rightarrow\ (z\ =\ x)))$

----

## Comments on the answers:

As previously mentioned, the translation of the sentence *"Nobody admires everybody"* depends on the domain of discourse. Now that we have a domain specified in the valuation, we can see that the more abstract version,  including all entities — evaluates to *False*, while the more restricted version  evaluates to *True*.

In regards to the sentence *Exactly one musician plays everything Alex wrote* we can observe that altough difference translation were given, all evaluate to *True* as  as they are semantically equivalent within the model.




## Lambda calculus

In [22]:
from nltk.grammar import FeatureGrammar

### 6. Function application and $\beta$-reduction
In the following examples some code has been deleted and replaced with `<????>`. What has been deleted? Verify that your answer is correct. **[4 marks]**

In [23]:
e1 = read_expr(r'\x.like(x, rob)')
e2 = read_expr(r'pip')
e3 = nltk.sem.ApplicationExpression(e1,e2)
display_latex(e3.simplify())
# with result like(pip,rob).
display_latex(read_expr(r"like(pip,rob)"))

e1 = read_expr(r'\P.P(pip)')
e2 = read_expr(r'\x.play(x,scherzo)')
e3 = nltk.sem.ApplicationExpression(e1,e2)
display_latex(e3.simplify())
# with result play(pip,scherzo).
display_latex(read_expr(r"play(pip,scherzo)"))

e1 = read_expr(r'\P.exists x.(woman(x) & P(x))')
e2 = read_expr(r'\x.play(x,etude)')
e3 = nltk.sem.ApplicationExpression(e1,e2)
display_latex(e3.simplify())
# with result exists x.(woman(x) & play(x,etude)).
display_latex(read_expr(r"exists x.(woman(x) & play(x,etude))"))

e1 = read_expr(r'\P x.P(\y.like(x,y))')
e2 = read_expr(r'\P.all x. (musician(x) -> P(x))')
e3 = nltk.sem.ApplicationExpression(e1,e2)
display_latex(e3.simplify())
# with result \x.all z2.(musician(z2) -> like(x,z2)).
display_latex(read_expr(r"\x.all z2.(musician(z2) -> like(x,z2))"))

$like(pip,rob)$

$like(pip,rob)$

$play(pip,scherzo)$

$play(pip,scherzo)$

$\exists\ x.(woman(x)\ \land\ play(x,etude))$

$\exists\ x.(woman(x)\ \land\ play(x,etude))$

$\lambda\ x.\forall\ z_{2}.(musician(z_{2})\ \rightarrow\ like(x,z_{2}))$

$\lambda\ x.\forall\ z_{2}.(musician(z_{2})\ \rightarrow\ like(x,z_{2}))$

### 7. Extending the grammar

Extend the grammar simple_sem.fcfg that comes with NLTK `(~/nltk_data/grammars/book_grammars/)` so that it will cover the following sentences:

- no man gives a bone to a dog **[4 marks]**
- no man gives a bone to the dog **[4 marks]**
- a boy and a girl chased every dog **[2 marks]**
- every dog chased a boy and a girl **[2 marks]**
- a brown cat chases a white dog **[4 marks]**

The last example includes adjectives. Several different kinds of adjectives are discussed in the literature [(cf. Kennedy, 2012)](http://semantics.uchicago.edu/kennedy/docs/routledge.pdf). In this example we have an intersective adjective. The denotiation we want for "brown cat" is a a set that we get by intersecting the set of individuals that are brown and the set of individuals that are cats.

C. Kennedy. Adjectives. In G. Russell, editor, The Routledge Companion to Philosophy of Language, chapter 3.3, pages 328–341. Routledge, 2012.

The original grammar is included in the code below as a string.

In [24]:
fcfg_string_orginal = r"""
% start S
############################
# Grammar Rules
#############################

S[SEM = <?subj(?vp)>] -> NP[NUM=?n,SEM=?subj] VP[NUM=?n,SEM=?vp]

NP[NUM=?n,SEM=<?det(?nom)> ] -> Det[NUM=?n,SEM=?det]  Nom[NUM=?n,SEM=?nom]
NP[LOC=?l,NUM=?n,SEM=?np] -> PropN[LOC=?l,NUM=?n,SEM=?np]

Nom[NUM=?n,SEM=?nom] -> N[NUM=?n,SEM=?nom]

VP[NUM=?n,SEM=?v] -> IV[NUM=?n,SEM=?v]
VP[NUM=?n,SEM=<?v(?obj)>] -> TV[NUM=?n,SEM=?v] NP[SEM=?obj]
VP[NUM=?n,SEM=<?v(?obj,?pp)>] -> DTV[NUM=?n,SEM=?v] NP[SEM=?obj] PP[+TO,SEM=?pp]

PP[+TO, SEM=?np] -> P[+TO] NP[SEM=?np]

#############################
# Lexical Rules
#############################

PropN[-LOC,NUM=sg,SEM=<\P.P(angus)>] -> 'Angus'
PropN[-LOC,NUM=sg,SEM=<\P.P(cyril)>] -> 'Cyril'
PropN[-LOC,NUM=sg,SEM=<\P.P(irene)>] -> 'Irene'

Det[NUM=sg,SEM=<\P Q.all x.(P(x) -> Q(x))>] -> 'every'
Det[NUM=pl,SEM=<\P Q.all x.(P(x) -> Q(x))>] -> 'all'
Det[SEM=<\P Q.exists x.(P(x) & Q(x))>] -> 'some'
Det[NUM=sg,SEM=<\P Q.exists x.(P(x) & Q(x))>] -> 'a'
Det[NUM=sg,SEM=<\P Q.exists x.(P(x) & Q(x))>] -> 'an'

N[NUM=sg,SEM=<\x.man(x)>] -> 'man'
N[NUM=sg,SEM=<\x.girl(x)>] -> 'girl'
N[NUM=sg,SEM=<\x.boy(x)>] -> 'boy'
N[NUM=sg,SEM=<\x.bone(x)>] -> 'bone'
N[NUM=sg,SEM=<\x.ankle(x)>] -> 'ankle'
N[NUM=sg,SEM=<\x.dog(x)>] -> 'dog'
N[NUM=pl,SEM=<\x.dog(x)>] -> 'dogs'

IV[NUM=sg,SEM=<\x.bark(x)>,TNS=pres] -> 'barks'
IV[NUM=pl,SEM=<\x.bark(x)>,TNS=pres] -> 'bark'
IV[NUM=sg,SEM=<\x.walk(x)>,TNS=pres] -> 'walks'
IV[NUM=pl,SEM=<\x.walk(x)>,TNS=pres] -> 'walk'
TV[NUM=sg,SEM=<\X x.X(\ y.chase(x,y))>,TNS=pres] -> 'chases'
TV[NUM=pl,SEM=<\X x.X(\ y.chase(x,y))>,TNS=pres] -> 'chase'
TV[NUM=sg,SEM=<\X x.X(\ y.see(x,y))>,TNS=pres] -> 'sees'
TV[NUM=pl,SEM=<\X x.X(\ y.see(x,y))>,TNS=pres] -> 'see'
TV[NUM=sg,SEM=<\X x.X(\ y.bite(x,y))>,TNS=pres] -> 'bites'
TV[NUM=pl,SEM=<\X x.X(\ y.bite(x,y))>,TNS=pres] -> 'bite'
DTV[NUM=sg,SEM=<\Y X x.X(\z.Y(\y.give(x,y,z)))>,TNS=pres] -> 'gives'
DTV[NUM=pl,SEM=<\Y X x.X(\z.Y(\y.give(x,y,z)))>,TNS=pres] -> 'give'

P[+to] -> 'to'
"""

Write your extension of this grammar here:

In [25]:

nltk.data.clear_cache()

fcfg_string = fcfg_string_orginal + r"""
## Your answers here

##########################
# Grammar Rules
##########################

NP[NUM=?n, SEM=<\P. ((?nom1(P) and ?nom2(P)))>] -> NP[NUM=?n, SEM=?nom1] CONJ NP[NUM=?n, SEM=?nom2]
NP[NUM=?n, SEM=<?det(?adj(?nom))>] -> Det[NUM=?n,SEM=?det] ADJ[NUM=?n,SEM=?adj] Nom[NUM=?n,SEM=?nom]


##########################
# Lexical Rules
##########################

Det[NUM=sg,SEM=<\P Q.-exists x.(P(x) & Q(x))>] -> 'no'
Det[NUM=sg,SEM=<\P Q.exists x.(P(x) -> Q(x))>] -> 'the'

CONJ -> 'and'
TV[SEM=<\X x.X(\ y.chase(x,y))>,TNS=past] -> 'chased'

N[NUM=sg,SEM=<\x.cat(x)>] -> 'cat'
ADJ[NUM=sg,SEM=<\P x.(P(x) & brown(x))>] -> 'brown'
ADJ[NUM=sg,SEM=<\P x.(P(x) & white(x))>] -> 'white'



"""

# Load `fcfg_string` as a feature grammar:
syntax = FeatureGrammar.fromstring(fcfg_string)



Run the code below without errors:

In [26]:
# comment out sentences if you couldn't find an answer for them
sentences = [
    'no man gives a bone to a dog',
    'no man gives a bone to the dog',
    'a boy and a girl chased every dog',
    'every dog chased a boy and a girl',
    'a brown cat chases a white dog',
]
for results in nltk.interpret_sents(sentences, syntax):
    for (synrep, semrep) in results:
        display(Markdown('----'))
        display_latex(semrep) # prints the SEM feature of a tree
        display_tree(synrep) # show the parse tree

----

$-\exists\ x.(man(x)\ \land\ \exists\ z_{4}.(dog(z_{4})\ \land\ \exists\ z_{3}.(bone(z_{3})\ \land\ give(x,z_{3},z_{4}))))$

TclError: no display name and no $DISPLAY environment variable

If you are working with iPython which is also running behind Jupyter notebooks and you are changing grammars and want to rerun a new version without restarting you may find `nltk.data.clear_cache()` useful.

## Statement of contribution



The work for this assignment was completed following a group meeting in which all members were present. Prior to the meeting, the group communicated and agreed that the best approach would be for each of us to attempt the exercises individually, and then meet later to discuss answers, share thoughts, and clarify any doubts or issues that arose.

Most of the exercises were solved in a similar way by all team members, with minor differences in the translations of ambiguous sentences. In such cases, we provided the different logical interpretations.

The answers in this draft represent a collaborative effort, where we compared our versions and included all alternatives when ambiguity led to differing interpretations.


## Marks

The assignment is marked on a 7-level scale where 4 is sufficient to complete the assignment; 5 is good solid work; 6 is excellent work, covers most of the assignment; and 7: creative work.

This assignment has a total of 47 marks. These translate to grades as follows: 1 = 17% 2 = 34%, 3 = 50%, 4 = 67%, 5 = 75%, 6 = 84%, 7 = 92% where %s are interpreted as lower bounds to achieve that grade.